In [1]:
# Do not edit this cell

# course: 3654
# a: Project 1
# d: VT

# Project 1: SNP 500 Companies Pre/Post Covid

**Name:**  Thejus Poruthikode Unnivelan
**PID:**  thejuspu

**Name:**  Andrew Visocan
**PID:**  avisocan
    
We have neither given nor received unauthorized assistance on this assignment. See the course sylabus for details on the Honor Code policy. In particular, sharing lines of solution code is prohibited.

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt